In [167]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from numpy.random import normal as normDist
from numpy import array

In [2]:
dirCats=r'/mnt/Data/Work/Sources/COSMOS/Catalogs'
fileCat=r'06_COSMOS_SpectrZ_Zlim_1p2_AllBands_Errors.csv'

In [3]:
KB=pd.read_csv(os.path.join(dirCats,fileCat))

In [32]:
KB=KB.rename({'Seq_1':'Seq'},axis=1)

In [33]:
print(len(KB))
print(KB.columns)

19176
Index(['Seq', 'e_Ksmagap3', 'e_Ymagap3', 'e_Hmagap3', 'e_Jmagap3', 'e_Bmagap3',
       'e_Vmagap3', 'e_ipmagap3', 'e_rmagap3', 'e_umagap3',
       ...
       'r/IA679', 'r/IA738', 'r/NB711', 'ip/IA624', 'ip/IA679', 'ip/IA738',
       'ip/IA767', 'ip/NB711', 'ip/NB816', 'zpp/NB816'],
      dtype='object', length=173)


In [109]:
nNewGals=5
mags=list([col for col in KB.columns if ('ap3' in col and 'e_' not in col)])
mags.remove('zpmagap3')
idCol='Seq'

In [149]:
generateMags = lambda magVals, magErrs,nNewGals: normDist(magVals,magErrs,(1,nNewGals,len(magVals)))

In [153]:
def generateGals(galVector,mags,idCol,nNewGals):
    errs=['e_'+mag for mag in mags]
    data=generateMags(galVector[mags],galVector[errs],nNewGals)
    galsNew=pd.DataFrame(data=data[0],columns=mags)
    galsNew[idCol]=galVector[idCol]
    idValsNew=[str(galVector[idCol])+'_'+str(i) for i in range(nNewGals)]
    galsNew[idCol+'_new']=idValsNew

    return galsNew

In [88]:
def augment(galVector,nNewGals,mags,idCol):
    df=pd.DataFrame(columns=list(galVector.index)+[idCol+'_new'])
    for i in range(nNewGals):
        df=df.append(generateGal(galVector,mags,idCol+'_new',idVal=str(galVector[idCol])+'_'+str(i+1)))
    return df

In [164]:
data=KB[:500]

In [165]:
%%time
df=pd.DataFrame(columns=list(data.columns)+[idCol+'_new'])
for row in data.iterrows():
    df=df.append(generateGals(row[1],mags,idCol,nNewGals),ignore_index=True)

CPU times: user 40.7 s, sys: 28.3 ms, total: 40.7 s
Wall time: 40.8 s


In [170]:
a=array.array()

AttributeError: 'builtin_function_or_method' object has no attribute 'array'

In [171]:
def generateGals(galVector,mags,idCol,nNewGals):
    errs=['e_'+mag for mag in mags]
    data=generateMags(galVector[mags],galVector[errs],nNewGals)
    galsNew=array(data=data[0],columns=mags)
    galsNew[idCol]=galVector[idCol]
    idValsNew=[str(galVector[idCol])+'_'+str(i) for i in range(nNewGals)]
    galsNew[idCol+'_new']=idValsNew

    return galsNew

In [172]:
data=KB[:10]


In [173]:
%%time
df=pd.DataFrame(columns=list(data.columns)+[idCol+'_new'])
for row in data.iterrows():
    df=df.append(generateGals(row[1],mags,idCol,nNewGals),ignore_index=True)

TypeError: array() missing required argument 'object' (pos 1)